In [1]:
from pyspark.sql.functions import hour,minute, second, col, avg, when
from pyspark.sql import SparkSession

In [2]:
spark= SparkSession.builder.appName("Learn").getOrCreate()
cpuDetails=spark.read.csv('usrINFO.csv',header=True, inferSchema=True)
cpuDetails.show()

+--------------------+-------------------+-------------------+-------------------+-------------------+
|           user_name|          idle_time|      working_hours|         start_time|           end_time|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|  sahil24c@gmail.com|2019-10-24 05:05:00|2019-10-24 05:50:00|2019-10-24 08:30:02|2019-10-24 19:25:02|
|magadum.iranna@gm...|2019-10-24 02:15:00|2019-10-24 08:39:59|2019-10-24 08:30:02|2019-10-24 19:25:01|
|  yathink3@gmail.com|2019-10-24 01:30:00|2019-10-24 09:24:59|2019-10-24 08:30:02|2019-10-24 19:25:01|
|  shelkeva@gmail.com|2019-10-24 00:30:00|2019-10-24 09:10:01|2019-10-24 08:45:01|2019-10-24 18:25:02|
|puruissimple@gmai...|2019-10-24 03:15:00|2019-10-24 07:19:59|2019-10-24 08:50:02|2019-10-24 19:25:01|
|sangita.awaghad19...|2019-10-24 01:55:00|2019-10-24 08:40:00|2019-10-24 08:50:01|2019-10-24 19:25:01|
|vaishusawant143@g...|2019-10-24 00:35:00|2019-10-24 09:55:00|2019-10-24 

In [16]:
dataWorkingHour=cpuDetails.withColumn('hours2Sec',hour(cpuDetails['working_hours'])*3600)
dataWorkingMin=cpuDetails.withColumn('Minute2Sec',minute(cpuDetails['working_hours'])*60)
dataWorkingSec=cpuDetails.withColumn('Seconds',second(cpuDetails['working_hours']))
hours=dataWorkingHour.select('user_name','hours2Sec')
Minutes=dataWorkingMin.select('user_name','Minute2Sec')
seconds=dataWorkingSec.select('user_name','Seconds')
hours.show(5)
Minutes.show(5)
seconds.show(5)

+--------------------+---------+
|           user_name|hours2Sec|
+--------------------+---------+
|  sahil24c@gmail.com|    18000|
|magadum.iranna@gm...|    28800|
|  yathink3@gmail.com|    32400|
|  shelkeva@gmail.com|    32400|
|puruissimple@gmai...|    25200|
+--------------------+---------+
only showing top 5 rows

+--------------------+----------+
|           user_name|Minute2Sec|
+--------------------+----------+
|  sahil24c@gmail.com|      3000|
|magadum.iranna@gm...|      2340|
|  yathink3@gmail.com|      1440|
|  shelkeva@gmail.com|       600|
|puruissimple@gmai...|      1140|
+--------------------+----------+
only showing top 5 rows

+--------------------+-------+
|           user_name|Seconds|
+--------------------+-------+
|  sahil24c@gmail.com|      0|
|magadum.iranna@gm...|     59|
|  yathink3@gmail.com|     59|
|  shelkeva@gmail.com|      1|
|puruissimple@gmai...|     59|
+--------------------+-------+
only showing top 5 rows



In [18]:
hourAmin=hours.join(Minutes, on=['user_name'], how='inner')
hourAmin.show(5)

+--------------------+---------+----------+
|           user_name|hours2Sec|Minute2Sec|
+--------------------+---------+----------+
|  sahil24c@gmail.com|    18000|      3000|
|magadum.iranna@gm...|    28800|      2340|
|  yathink3@gmail.com|    32400|      1440|
|  shelkeva@gmail.com|    32400|       600|
|puruissimple@gmai...|    25200|      1140|
+--------------------+---------+----------+
only showing top 5 rows



In [20]:
hourAminAsec=hourAmin.join(seconds,on=['user_name'],how='inner')
hourAminAsec.show(5)

+--------------------+---------+----------+-------+
|           user_name|hours2Sec|Minute2Sec|Seconds|
+--------------------+---------+----------+-------+
|  sahil24c@gmail.com|    18000|      3000|      0|
|magadum.iranna@gm...|    28800|      2340|     59|
|  yathink3@gmail.com|    32400|      1440|     59|
|  shelkeva@gmail.com|    32400|       600|      1|
|puruissimple@gmai...|    25200|      1140|     59|
+--------------------+---------+----------+-------+
only showing top 5 rows



In [22]:
val=hourAminAsec.withColumn('Avgvalue', (hourAminAsec['hours2Sec']+hourAminAsec['Minute2Sec']+hourAminAsec['Seconds'])/3600)
val.show(5)

+--------------------+---------+----------+-------+-----------------+
|           user_name|hours2Sec|Minute2Sec|Seconds|         Avgvalue|
+--------------------+---------+----------+-------+-----------------+
|  sahil24c@gmail.com|    18000|      3000|      0|5.833333333333333|
|magadum.iranna@gm...|    28800|      2340|     59| 8.66638888888889|
|  yathink3@gmail.com|    32400|      1440|     59| 9.41638888888889|
|  shelkeva@gmail.com|    32400|       600|      1|9.166944444444445|
|puruissimple@gmai...|    25200|      1140|     59|7.333055555555555|
+--------------------+---------+----------+-------+-----------------+
only showing top 5 rows



In [23]:
higestAvgVal=val.select(avg(val[valuew'Avgvalue']))
higestAvgVal.show()

+-----------------+
|    avg(Avgvalue)|
+-----------------+
|7.137449494949498|
+-----------------+



In [25]:
count=val.filter(val['Avgvalue']>7.137).select('user_name','Avgvalue').show()

+--------------------+------------------+
|           user_name|          Avgvalue|
+--------------------+------------------+
|magadum.iranna@gm...|  8.66638888888889|
|  yathink3@gmail.com|  9.41638888888889|
|  shelkeva@gmail.com| 9.166944444444445|
|puruissimple@gmai...| 7.333055555555555|
|sangita.awaghad19...| 8.666666666666666|
|vaishusawant143@g...| 9.916666666666666|
|     you@example.com| 8.083055555555555|
|samadhanmahajan73...|  8.66638888888889|
|ashutoshrit64@gma...|10.416944444444445|
|akshaybavalekar10...| 8.583333333333334|
|khairnarswapna99@...| 9.416666666666666|
|kukadeshilpaa7m95...| 9.666666666666666|
|sarikabarge111@gm...|              8.75|
|narsimharaj.kasu0...| 8.083333333333334|
|antonyalexcm@gmai...|             10.25|
|jitupatil937@gmai...| 9.333333333333334|
|akshaypatwari24@g...| 9.083333333333334|
|aheteshams007@gma...|  8.83361111111111|
|surajpj7852@gmail...| 7.916666666666667|
|kaleshwetaanil@gm...| 9.499722222222223|
+--------------------+------------